In [7]:
#librerias
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import json
import pandas as pd
from datetime import datetime
import numpy as np
import re
from selenium.common.exceptions import TimeoutException

In [8]:
##Nuevas
def buscar(elemento,busqueda):
    WebDriverWait(driver, 10)\
        .until(EC.element_to_be_clickable((By.CSS_SELECTOR,elemento)))\
        .send_keys(busqueda)
    
def Click(elemento):
    WebDriverWait(driver, 5)\
        .until(EC.element_to_be_clickable((By.CSS_SELECTOR,elemento))).click()

def clear_buscar(elemento):
    WebDriverWait(driver, 20)\
        .until(EC.element_to_be_clickable((By.CSS_SELECTOR,elemento)))\
        .clear()
    
#Se solicita el Xfullpath para hacer click
def Click_Xpath(path):
    WebDriverWait(driver, 15)\
    .until(EC.element_to_be_clickable((By.XPATH,path)))\
    .click()
    
    
##Funcion NoClick hace referencia a los apartados que no son clikeables y se accede a el por medio del FullXpath
def NoClick(path):
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,path)))
    url=driver.find_element("xpath",path).get_attribute("href")
    driver.get(url)   
    
def ListarNombres(path,clase):
    WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.XPATH,path)))
    html_list=driver.find_element(By.CSS_SELECTOR,clase)
    lista = html_list.find_elements(By.TAG_NAME,'div')
    titulo=[]
    for i in lista:
        text = i.get_attribute('innerHTML')
        soup = BeautifulSoup(text,'html.parser')
        titulo.append(soup.findAll("h2")[0].string)
        #print(titulo)
    return titulo

def ListarPrecios(path,clase):
    WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.XPATH,path)))
    html_list=driver.find_element(By.CSS_SELECTOR,clase)
    lista = html_list.find_elements(By.TAG_NAME,'li')
    
    for i in lista:
        text = i.get_attribute('innerHTML')
        soup = BeautifulSoup(text,'html.parser')
        precio = soup.select("span.price-tag-fraction")[0].text
        print(precio)
def LeerJson(path):
    
    WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.XPATH,path)))
    Json_Jumbo=driver.find_element("xpath",path).get_attribute("text")
    Json_Jumbo = json.loads(Json_Jumbo)
    DatosProducto = Json_Jumbo['itemListElement'] # Se toma el Json anidado que contiene los datos del producto
    return DatosProducto

In [9]:
#Funciones para limpieza y procesamiento de datos 
def cantidad(producto):
    
    l=len(producto)
    producto=producto[::-1] #Invertimos la cadena para encontrar x
    m =producto.find("x")
    producto = producto[:m] #Esta invertimo el valor
    producto=producto[::-1] #Lo volvemos a su estado normal
    return producto
    
#Funcion para convertir palabras con acentos en palabras sin acentos
def categoria(producto):
    
    m =producto.find(" ")
    remplazo = producto[:m].maketrans("áéíóú", "aeiou","*123456789") #Remplazo palabras que contengan acentos y borrar caracteres *123456789
    producto = producto[:m].translate(remplazo).upper() #Uso translate para poder observar correctamente
    return producto



In [10]:
#Ruta de chorme compatible 
driver_path = "C:\\Users\\danie\\OneDrive\\Documentos\\Notebooks\\chromedriver.exe"
## Opciones con las que inicia google chrome 
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument("--headless")
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

s = Service(driver_path)#Se agrega el path como objeto para evitar errores 


# Main

## Función buscar:
- Recibe como parametro el input class del cuadro de busqueda de la pagina y el producto a buscar
- Se antepone input antes de colocar el input class
- Si el input class tiene espacios estos se remplazan por puntos

## Funcion Click
- Recibe como parametro el elemento clikeable anteponiendo el formato(div, footer, etc)


## Función ListarProductos
- Esta funcion lista todos los productos de la busquema realizada 


In [81]:
'//script[type="application/ld+json"]'

def LeerJson2(path):
    
    
    Json_Jumbo =driver.find_element("xpath",path).get_attribute("text")
    #Json_Jumbo = json.loads(Json_Jumbo)
    #DatosProducto = Json_Jumbo['itemListElement'] # Se toma el Json anidado que contiene los datos del producto
    return Json_Jumbo


def LeerJson3(path):
    
    WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.XPATH,path)))
    Json_Jumbo=driver.find_element("xpath",path).get_attribute("text")
    #Json_Jumbo = json.loads(Json_Jumbo)
    #DatosProducto = Json_Jumbo['itemListElement'] # Se toma el Json anidado que contiene los datos del producto
    return DatosProducto

In [89]:
driver = webdriver.Chrome(service=s)
driver.maximize_window()
driver.get("https://www.tiendasjumbo.co/") 

ListaProducto=  ['Arroz']
CantMas = dict([('Arroz', 1)])

#driver.implicitly_wait (10) 
Json_Total=[]
time.sleep(1)

try:
    for i in range(len(ListaProducto)):
        time.sleep(1)
    #Buscar producto
        producto = ListaProducto[i]
        input_class = 'input.vtex-styleguide-9-x-input.ma0.border-box.vtex-styleguide-9-x-hideDecorators.vtex-styleguide-9-x-noAppearance.br2.br-0.br--left.w-100.bn.outline-0.bg-base.c-on-base.b--muted-4.hover-b--muted-3.t-body.pl5'
        buscar(input_class,producto)
        time.sleep(2)
    #Click en buscar
        Element_clickeable = 'footer.vtex-search-2-x-tileListFooter'
        Click(Element_clickeable)
        time.sleep(3)
        driver.switch_to.default_content()
    #Guardar en Json     
        time.sleep(1)
        driver.set_page_load_timeout(30)
               
        path= '/html/body/div[2]/div/div[1]/div/div[2]/div/div/section/div[2]/div/div[9]/section/div/div/script'
        Json_Total += LeerJson(path) #Esto deja un JSON con el que se trabajara los datos
    #limpiar
        if(i>=1):
            driver.get("https://www.tiendasjumbo.co/") 

except TimeoutException:
    driver.set_page_load_timeout(10)
    
    print('El proceso finalizo con algo inusual')
print('Finalizo correctamente el proceso')

El proceso finalizo con algo inusual
Finalizo correctamente el proceso


In [90]:
Json_Total

[]

# Procesamiento de datos

In [12]:
#Se obtienen los datos del segundo nivel de profundidad del json y se pasan a un dataframe
Producto_1 = pd.json_normalize(Json_Total) 
#Se accede al 3 nivel de produndidad del json 
JsonAnidado=[] #Se crea una lista vacia para llenarla con el for

for i in Producto_1['item.offers.offers']:
    JsonAnidado +=i 
    
#Se normaliza el nivel 3 de profundidad del json    
Producto_2= pd.json_normalize(JsonAnidado) # Hacemos join de los 2 dataframe creados
Productos_T = pd.concat([Producto_1, Producto_2], axis = 1) 

In [13]:
#Seleccionamos las columnas que nos interesan
Productos_T = Productos_T[['sku','item.name','item.brand.name','item.description','priceCurrency','price','priceValidUntil','item.@id','item.image','seller.name']]
#Renombramos las columnas 
Productos_T.columns = ['Codigo_Pro','Nombre', 'Marca', 'Descripcion', 'Moneda', 'Precio','Precio_Valido_Hasta','Url_producto', 'Url_Imagen','Almacen']

In [14]:
#Obtenemos la fecha actual
now = datetime.now()
format = now.strftime('%Y-%m-%d')
NombreAlmacen='Jumbo-'+format

In [15]:
#Funcion para obtener la cantidad de cada producto
def cantidadx(producto):
    
    producto = re.findall(r'( x.+\d{0,6}.\w{1,3}|\d{1,5}..\w{0,5}| \d{1,3}\D\d{1,3}.\w{1,3})',producto) #Expresion regular para obtener las cantidades de los productos
    producto = " ".join(producto).replace('x', '') #Eliminamos la x en la cantidad de productos y pasamos los datos a str

    #----Remplazar valores inusuales----
    producto = re.sub(' \.', '.',producto) #Eliminamos espacios vacios en valores decimales
    

    
    return producto #Retornar en minusculas y sin espacios al inicio y al final 

#Función para separar el numero de la unidad de medida en el nombre
def separador(producto):
    patron =r'(\d{1,5})([a-z]{0,7})' #Expresion regular para dar espacio entre numeros y letras

    producto = re.sub(patron,r'\1 \2',producto) #Separamos los numeros que estan junto a palabras de unidad de medida
    producto = re.sub(' +', ' ',producto) #Esta linea solo deja un espacio entre palabras eliminando espacios inecesarios
    producto = re.sub(' \,', '.',producto) #Eliminamos espacios vacios en valores decimales
    producto = re.sub(' \.', '.',producto) #Eliminamos espacios vacios en valores decimales
    producto = producto.replace('&amp;co ', ' ')
    producto = producto.replace('-', '')

    return producto.lower()


def EliminarInusuales(producto):
    
    #Primero se remplaza valores de str que son inusuales
    producto = producto.replace('con vita', '')
    producto = producto.replace('van camps', '')
    producto = producto.replace('el rey', '')


    #Luego normalizamos las medidas 
    producto = producto.replace('1000 g', '1 kg')
    producto = producto.replace('2000 g', '2 kg')
    producto = producto.replace('3000 g', '3 kg')
    producto = producto.replace('4000 g', '4 kg')
    producto = producto.replace('5000 g', '5 kg')
    producto = producto.replace('10000 g', '10 kg')
    producto = producto.replace('1000 mlb', '500 g')

    #Para unidades liquidas

    producto = producto.replace('lt', 'l')
    producto = producto.replace('cc', 'ml')
    producto = producto.replace('cm3', 'ml')
    producto = producto.replace('cm3', 'ml')
    producto = producto.replace('1 l', '1000 ml')
    producto = producto.replace('1.8 l', '1800 ml')
    producto = producto.replace('3 l', '3000 ml')






    producto = producto.replace('kilo', 'kg')
    producto = producto.replace('kgs', 'kg')
    producto = producto.replace('2500 g', '2.5 kg')

    producto = re.sub(r"gratis.\w{1,5}.\w{1,20}","",producto)

    return producto.strip()


# Limpieza de datos

In [16]:

Productos_T['Nombre']=Productos_T['Nombre'].apply(lambda x: separador(x))
Productos_T['Cantidad']=Productos_T['Nombre'].apply(lambda x: cantidadx(x))
Productos_T['Fecha']= now.strftime('%Y-%m-%d')
Productos_T['Categoria']=Productos_T['Nombre'].apply(lambda x: categoria(x))
Productos_T['Cantidad']=Productos_T['Cantidad'].apply(lambda x: EliminarInusuales(x))

## Exporta archivo json


In [17]:
#Exportar archivo JSON
Productos_T.to_json(f'{NombreAlmacen}.json', orient = 'split', compression = 'infer', index = 'true') 

In [18]:
#Productos_T.to_csv(f'{NombreAlmacen}.csv', index=False,sep='|',encoding='utf-8')

In [19]:
Productos_T['Cantidad']

0                10 kg
1     25 und  500 g cu
2                10 kg
3                10 kg
4                 5 kg
5                 5 kg
6                 3 kg
7                 1 kg
8                 5 kg
9                10 kg
10                5 kg
11     25 und 500 g cu
12                4 kg
13                1 kg
14               10 kg
15                5 kg
16               10 kg
17                3 kg
18                3 kg
19              2.5 kg
20              2.5 kg
21                5 kg
22                1 kg
23                4 kg
Name: Cantidad, dtype: object

In [20]:


# Ver todo el contenido de un dataframe
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.width = None
pd.options.display.max_colwidth = None

In [21]:
m = Productos_T[['Nombre','Cantidad']]
m

,Nombre,Cantidad
0,arroz vitarroz plus supremo x 10 kg,10 kg
1,arroz diana x 25 und x 500 g cu,25 und 500 g cu
2,arroz cuisine&amp;co blanco x10000 g,10 kg
3,arroz diana x 10 kg,10 kg
4,arroz diana x 5 kg,5 kg
5,arroz supremo x 5 kg,5 kg
6,arroz diana x 3 kg,3 kg
7,arroz diana x 1 kg,1 kg
8,arroz cuisine&amp;co blanco x5000 g,5 kg
9,arroz roa x 10 kg,10 kg


In [22]:
#Productos_T['Nombre']